This tutorial shows the potential of using CO2 capture resources in Macro such as Natural Gas Power CCS, SMR H2 CCS, and direct air capture (DAC). Natural Gas Power CCS and SMR H2 CCS works by specifying a CO2 capture rate (for example 0.9) of the total CO2 emitted by natural gas used in producing power and H2 which effectively reduces the total emissions by 90%. On the other hand, DAC leads to a true removal of CO2 from the atmosphere and not just a reduction of emissions. 

In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\pecci\Code\MACRO-dev\tutorials`


In [2]:
using Macro,CSV,DataFrames

Start MACRO model generation (units: MWh for energy, $ for costs)

In [3]:
T = 8760;
macro_settings = (Commodities = Dict(Electricity=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    Hydrogen=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    NaturalGas=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    CO2=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T)),
                PeriodLength = T);

In [4]:
macro_settings

(Commodities = Dict{DataType, Dict{Symbol, Int64}}(Hydrogen => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), CO2 => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), Electricity => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), NaturalGas => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1)), PeriodLength = 8760)

In [5]:

H2_MWh = 33.33 # MWh per tonne of H2
NG_MWh = 0.29307107 # MWh per MMBTU of NG

df = CSV.read("time_series_data.csv",DataFrame)

electricity_demand = df[1:T,:Electricity_Demand_MW]; # MWh
solar_capacity_factor = df[1:T,:Solar_Capacity_Factor]; # factor between 0 and 1
ng_fuel_price = df[1:T,:NG_Price]/NG_MWh; # $/MWh of natural gas
h2_demand = H2_MWh*df[1:T,:H2_Demand_tonne] # MWh of hydrogen

solar_inv_cost = 85300; # $/MW
solar_fom_cost = 18760.0; # $/MW

battery_inv_cost = 19584.0; #$/MW
battery_fom_cost = 4895; # $/MW
battery_vom_cost = 0.15; #$/MW
battery_inv_cost_storage = 22494.0; #$/MWh
battery_fom_cost_storage = 5622; #$/MWh
battery_vom_cost_storage = 0.15; #$/MWh
battery_eff_up = 0.92;
battery_eff_down = 0.92;

ngcc_inv_cost = 65400;# $/MW
ngcc_fom_cost = 10287.0;# $/MW
ngcc_vom_cost = 3.55; #$/MW
ngcc_capsize = 250.0;
ngcc_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of electricity
ngcc_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas

ngcc_ccs_inv_cost = 66400;# $/MW
ngcc_ccs_fom_cost = 12287.0;# $/MW
ngcc_ccs_vom_cost = 3.65; #$/MW
ngcc_ccs_capsize = 250.0;
ngcc_ccs_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of electricity
ngcc_ccs_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas
ngcc_ccs_CO2_captured_rate = 0.9; #Tons of CO2 captured/Tons of CO2 produced by fuel

electric_dac_inv_cost = 9390;# $/Tonnes of CO2 captured/h
electric_dac_fom_cost = 7470;# $/Tonnes of CO2 captured/h
electric_dac_vom_cost = 22; #$/Tonnes of CO2 captured
electric_dac_power_input = 4.38; #MWh/Tonnes of CO2 captured
electric_dac_capsize = 20;# Tonnes of CO2 captured/h

electrolyzer_capsize = 2*H2_MWh # MWh of H2
electrolyzer_efficiency = 1/(45/H2_MWh) # MWh of H2 / MWh of electricity
electrolyzer_inv_cost = 2033333/H2_MWh # $/MW of H2
electrolyzer_fom_cost = 30500/H2_MWh # $/MW of H2
electrolyzer_vom_cost = 0.0;

smr_inv_cost = 1033333/H2_MWh;# $/MW of H2
smr_fom_cost = 20500/H2_MWh;# $/MW of H2
smr_vom_cost = 0.0; #$/MW of H2
smr_capsize = 20*H2_MWh;
smr_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of H2
smr_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas

smr_ccs_inv_cost = 1133333/H2_MWh;# $/MW of H2
smr_ccs_fom_cost = 21500/H2_MWh;# $/MW of H2
smr_ccs_vom_cost = 0.0; #$/MW of H2
smr_ccs_capsize = 20*H2_MWh;
smr_ccs_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of H2
smr_ccs_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas
smr_ccs_CO2_captured_rate = 0.9; #Tons of CO2 captured/Tons of CO2 produced by fuel


In [6]:
### For now, we use the same temporal resolution and subperiods for every commodity, but MACRO will allow to use different temporal resolution for different commodities

hours_per_timestep(c) = macro_settings.Commodities[c][:HoursPerTimeStep];
hours_per_subperiod(c) = macro_settings.Commodities[c][:HoursPerSubperiod]
time_interval(c)= 1:hours_per_timestep(c):macro_settings.PeriodLength

subperiods(c) = collect(Iterators.partition(time_interval(c), Int(hours_per_subperiod(c) / hours_per_timestep(c))),)

subperiods (generic function with 1 method)

Let's start from creating an electricity node:

In [7]:
e_node = Node{Electricity}(;
    id = Symbol("E_node"),
    demand =  electricity_demand,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    max_nsd = [0.0],
    price_nsd = [50000],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We assign a solar PV generator to this electricity node:

In [8]:
solar_pv = Resource{Electricity}(;
    id = :Solar_PV,
    node = e_node,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    capacity_factor = solar_capacity_factor,
    can_expand = true, 
    can_retire = false,
    existing_capacity = 0.0,
    investment_cost = solar_inv_cost,
    fixed_om_cost = solar_fom_cost,
    constraints = [Macro.CapacityConstraint()]
)

Resource{Electricity}(Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :Solar_PV, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1779, 0.429  …  0.3834, 0.2594, 0.078, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1.0, Float64[], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], 0.0, Inf, 0.0, true, false, 85300.0, 18760.0, 0.0, Dict{Symbol, Any}(), Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])

We can also assigne a battery storage to the node:

In [9]:
battery = SymmetricStorage{Electricity}(;
    id = :battery,
    node = e_node,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    can_expand = true,
    can_retire = false,
    existing_capacity = 0.0,
    investment_cost = battery_inv_cost,
    investment_cost_storage = battery_inv_cost_storage,
    fixed_om_cost = battery_fom_cost,
    fixed_om_cost_storage = battery_fom_cost_storage,
    variable_om_cost = battery_vom_cost,
    variable_om_cost_storage = battery_vom_cost_storage,
    efficiency_injection = battery_eff_down,
    efficiency_withdrawal = battery_eff_up,
    constraints = [Macro.CapacityConstraint(),Macro.StorageCapacityConstraint()]
)

SymmetricStorage{Electricity}(Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :battery, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 0.0, Inf, 0.0, Inf, 0.0, 0.0, true, false, 19584.0, 22494.0, 0.0, 4895.0, 5622.0, 0.0, 0.15, 0.15, 0.0, 0.92, 0.92, 0.0, 1.0, 10.0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing), Macro.StorageCapacityConstraint(missing, missing, missing)])

We also consider a hydrogen demand node:

In [10]:
h2_node = Node{Hydrogen}(;
    id = Symbol("H2_node"),
    demand =  h2_demand,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We model an exogenous inflow of natural gas into the system using by introducing a natural gas source node (with zero demand):

In [11]:
ng_node = Node{NaturalGas}(;
    id = Symbol("NG_node"),
    time_interval = time_interval(NaturalGas),
    subperiods = subperiods(NaturalGas),
    demand = zeros(length(time_interval(NaturalGas))),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

ng_source = Resource{NaturalGas}(;
id = :NG_Source,
node = ng_node,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
can_expand = false, 
can_retire = false,
price = ng_fuel_price,
existing_capacity = Inf,
)

Resource{NaturalGas}(Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :NG_Source, Float64[], 1.0, [18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264  …  14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], 0.0, Inf, Inf, false, false, 0.0, 0.0, 0.0, Dict{Symbol, Any}(), Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[])

We also define a CO2 node with both a sink and source:

In [12]:
co2_node = Node{CO2}(;
id = Symbol("CO2_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
demand = zeros(length(time_interval(CO2))),
max_nsd = [0.0],
price_nsd = [0.0],
constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

co2_sink = Sink{CO2}(;
node = co2_node,
id = Symbol("CO2_sink"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
)

co2_source = Resource{CO2}(;
id = :CO2_Source,
node = co2_node,
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
can_expand = false, 
can_retire = false,
price = zeros(length(time_interval(CO2))),
existing_capacity = Inf,
)

Resource{CO2}(Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :CO2_Source, Float64[], 1.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], 0.0, Inf, Inf, false, false, 0.0, 0.0, 0.0, Dict{Symbol, Any}(), Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[])

We define a captured CO2 node and sink to account for the inventory of captured CO2 (input from CO2 capture, output to SNG or potential storage):

In [13]:
co2_captured_node = Node{CO2}(;
id = Symbol("CO2_captured_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
demand = zeros(length(time_interval(CO2))),
max_nsd = [0.0],
price_nsd = [0.0],
constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

co2_captured_sink = Sink{CO2}(;
node = co2_captured_node,
id = Symbol("CO2_captured_sink"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
)

Sink{CO2}(Node{CO2}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :CO2_captured_sink, Float64[], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[])

We are now ready to define a natural gas plant! Note that we require two stoichiometry balances:

Heat_Rate $\times$ Electricity = NaturalGas

Fuel_CO2 $\times$ NaturalGas = CO2

This transformation has 3 edges: each edge carries a specific commodity flow

In [14]:
ngcc = Transformation{NaturalGasPower}(;
id = :NGCC,
time_interval = time_interval(Electricity),
stoichiometry_balance_names = [:energy,:emissions],
constraints = [Macro.StoichiometryBalanceConstraint()]
)

ngcc.TEdges[:E] = TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = ngcc,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
capacity_size = ngcc_capsize,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
st_coeff = Dict(:energy=>ngcc_heatrate,:emissions=>0.0),
existing_capacity = 0.0,
investment_cost = ngcc_inv_cost,
fixed_om_cost = ngcc_fom_cost,
variable_om_cost = ngcc_vom_cost,
constraints = [CapacityConstraint()]
)

ngcc.TEdges[:NG] = TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = ngcc,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = Dict(:energy=>1.0,:emissions=>ngcc_fuel_CO2)
)

ngcc.TEdges[:CO2] = TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = ngcc,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>1.0)
)

TEdge{CO2}(:CO2, Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPower}(:NGCC, 1:1:8760, [:energy, :emissions], Dict{Symbol, TEdge}(:NG => TEdge{NaturalGas}(:NG, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPower}(#= circular reference @-3 =#), :input, false, false, false, 1.0, 1:1:8760, St

We are now ready to define a natural gas plant with CCS! Note that we require three stoichiometry balances:

Heat_Rate $\times$ Electricity = NaturalGas

Fuel_CO2 $\times$ NaturalGas $\times$ (1-CCS rate) = CO2

Fuel_CO2 $\times$ NaturalGas $\times$ CCS rate = CO2_Captured

This transformation has 4 edges: each edge carries a specific commodity flow

In [15]:
ngcc_ccs = Transformation{NaturalGasPowerCCS}(;
id = :NGCC_CCS,
time_interval = time_interval(Electricity),
stoichiometry_balance_names = [:energy,:emissions,:captured_emissions],
constraints = [Macro.StoichiometryBalanceConstraint()]
)

ngcc_ccs.TEdges[:E] = TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = ngcc_ccs,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
capacity_size = ngcc_ccs_capsize,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
st_coeff = Dict(:energy=>ngcc_ccs_heatrate,:emissions=>0.0,:captured_emissions=>0.0),
existing_capacity = 0.0,
investment_cost = ngcc_ccs_inv_cost,
fixed_om_cost = ngcc_ccs_fom_cost,
variable_om_cost = ngcc_ccs_vom_cost,
constraints = [CapacityConstraint()]
)

ngcc_ccs.TEdges[:NG] = TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = ngcc_ccs,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = Dict(:energy=>1.0,:emissions=>ngcc_ccs_fuel_CO2*(1-ngcc_ccs_CO2_captured_rate),:captured_emissions=>ngcc_ccs_fuel_CO2*ngcc_ccs_CO2_captured_rate)
)

ngcc_ccs.TEdges[:CO2] = TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = ngcc_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>1.0,:captured_emissions=>0.0)
)

ngcc_ccs.TEdges[:CO2_Captured]=TEdge{CO2}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = ngcc_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>0.0,:captured_emissions=>1.0)
)

TEdge{CO2}(:CO2_Captured, Node{CO2}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPowerCCS}(:NGCC_CCS, 1:1:8760, [:energy, :emissions, :captured_emissions], Dict{Symbol, TEdge}(:CO2_Captured => TEdge{CO2}(#= circular reference @-3 =#), :NG => TEdge{NaturalGas}(:NG, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Tran

We can also create an electrolyzer, which is defined by a single stoichiometry balance: electrolyzer_efficiency $\times$ Electricity = H2

In [16]:
electrolyzer = Transformation{Electrolyzer}(;
                id = :Electrolyzer,
                time_interval = time_interval(Electricity),
                stoichiometry_balance_names = [:energy],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

electrolyzer.TEdges[:H2] = TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = electrolyzer,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electrolyzer_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = Dict(:energy=>1.0),
    existing_capacity = 0.0,
    investment_cost = electrolyzer_inv_cost,
    fixed_om_cost = electrolyzer_fom_cost,
    variable_om_cost = electrolyzer_vom_cost,
    constraints = [CapacityConstraint()]
)

electrolyzer.TEdges[:E] = TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electrolyzer,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = Dict(:energy=>electrolyzer_efficiency)
)

TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electrolyzer}(:Electrolyzer, 1:1:8760, [:energy], Dict{Symbol, TEdge}(:H2 => TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(mi

We are now ready to define a natural gas hydrogen plant or SMR similar to a natural gas power plant

In [17]:
smr = Transformation{NaturalGasHydrogen}(;
                id = :SMR,
                time_interval = time_interval(Hydrogen),
                stoichiometry_balance_names = [:energy,:emissions],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

smr.TEdges[:H2] = TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = smr,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = smr_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = Dict(:energy=>smr_heatrate,:emissions=>0.0),
    existing_capacity = 0.0,
    investment_cost = smr_inv_cost,
    fixed_om_cost = smr_fom_cost,
    variable_om_cost = smr_vom_cost,
    constraints = [CapacityConstraint()]
)

smr.TEdges[:NG] = TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = smr,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = Dict(:energy=>1.0,:emissions=>smr_fuel_CO2)
)

smr.TEdges[:CO2] = TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = smr,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>1.0)
)

TEdge{CO2}(:CO2, Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasHydrogen}(:SMR, 1:1:8760, [:energy, :emissions], Dict{Symbol, TEdge}(:NG => TEdge{NaturalGas}(:NG, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasHydrogen}(#= circular reference @-3 =#), :input, false, false, false, 1.0, 1:1:876

Now, let's add a SMR hydrogen plant with CCS, similar to a natural gas power plant with CCS

In [18]:
smr_ccs = Transformation{NaturalGasHydrogenCCS}(;
                id = :SMR_CCS,
                time_interval = time_interval(Hydrogen),
                stoichiometry_balance_names = [:energy,:emissions,:captured_emissions],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

smr_ccs.TEdges[:H2] = TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = smr_ccs_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = Dict(:energy=>smr_ccs_heatrate,:emissions=>0.0,:captured_emissions=>0.0),
    existing_capacity = 0.0,
    investment_cost = smr_ccs_inv_cost,
    fixed_om_cost = smr_ccs_fom_cost,
    variable_om_cost = smr_ccs_vom_cost,
    constraints = [CapacityConstraint()]
)

smr_ccs.TEdges[:NG] = TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = smr_ccs,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = Dict(:energy=>1.0,:emissions=>smr_ccs_fuel_CO2*(1-smr_ccs_CO2_captured_rate),:captured_emissions=>smr_ccs_fuel_CO2*smr_ccs_CO2_captured_rate)
)

smr_ccs.TEdges[:CO2] = TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>1.0,:captured_emissions=>0.0)
)

smr_ccs.TEdges[:CO2_Captured] = TEdge{CO2}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>0.0,:captured_emissions=>1.0)
)

TEdge{CO2}(:CO2_Captured, Node{CO2}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasHydrogenCCS}(:SMR_CCS, 1:1:8760, [:energy, :emissions, :captured_emissions], Dict{Symbol, TEdge}(:CO2_Captured => TEdge{CO2}(#= circular reference @-3 =#), :NG => TEdge{NaturalGas}(:NG, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Tr

Let's add a direct air capture unit consuming electricity and capturing CO2 from the atmosphere into the captured CO2 inventory

In [19]:
electric_dac = Transformation{DACElectric}(;
                id = :Electric_DAC,
                time_interval = time_interval(CO2),
                stoichiometry_balance_names  = [:energy,:captured_co2],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

electric_dac.TEdges[:CO2_Captured] = TEdge{CO2}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = electric_dac,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electric_dac_capsize,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>electric_dac_power_input,:captured_co2=>1.0),
    existing_capacity = 0.0,
    investment_cost = electric_dac_inv_cost,
    fixed_om_cost = electric_dac_fom_cost,
    variable_om_cost = electric_dac_vom_cost,
    constraints = [CapacityConstraint()]
)

electric_dac.TEdges[:CO2_input] = TEdge{CO2}(;
    id = :CO2_input,
    node = co2_node,
    transformation = electric_dac,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:captured_co2=>1.0)
)

electric_dac.TEdges[:E] = TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electric_dac,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = Dict(:energy=>1.0,:captured_co2=>0.0)
)

TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{DACElectric}(:Electric_DAC, 1:1:8760, [:energy, :captured_co2], Dict{Symbol, TEdge}(:CO2_Captured => TEdge{CO2}(:CO2_Captured, Node{CO2}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Trans

Now that we have implemented different technologies, we can create the JuMP model.

In [20]:
components = [solar_pv; battery; ngcc; ngcc_ccs; electrolyzer; smr; smr_ccs; electric_dac;ng_source;co2_source;co2_sink;co2_captured_sink];
nodes = [e_node; h2_node; ng_node; co2_node; co2_captured_node];
system = [nodes;components];

In [21]:
model = Macro.JuMP.Model();
Macro.@variable(model,vREF==1) ## Variable used to initialize empty expressions
Macro.@expression(model, eFixedCost, 0 * model[:vREF]);
Macro.@expression(model, eVariableCost, 0 * model[:vREF]);

In [22]:
add_planning_variables!.(components,model);

In [23]:
add_operation_variables!.(system, model);

In [24]:
add_all_model_constraints!.(system, model);

In [25]:
Macro.@objective(model, Min, model[:eFixedCost] + model[:eVariableCost]);


In [26]:
println(ngcc.constraints[1].constraint_ref[:energy,5])

-vFLOW_NGCC_NG[5] + 2.1775180500999998 vFLOW_NGCC_E[5] == 0


In [27]:
println(ngcc.constraints[1].constraint_ref[:emissions,5])

-0.181048235160161 vFLOW_NGCC_NG[5] + vFLOW_NGCC_CO2[5] == 0


In [28]:
println(ngcc_ccs.constraints[1].constraint_ref[:energy,5])

-vFLOW_NGCC_CCS_NG[5] + 2.1775180500999998 vFLOW_NGCC_CCS_E[5] == 0


In [29]:
println(ngcc_ccs.constraints[1].constraint_ref[:emissions,5])

-0.018104823516016097 vFLOW_NGCC_CCS_NG[5] + vFLOW_NGCC_CCS_CO2[5] == 0


In [30]:
println(ngcc_ccs.constraints[1].constraint_ref[:captured_emissions,5])

vFLOW_NGCC_CCS_CO2_Captured[5] - 0.16294341164414491 vFLOW_NGCC_CCS_NG[5] == 0


In [31]:
println(electrolyzer.constraints[1].constraint_ref[:energy,5])

vFLOW_Electrolyzer_H2[5] - 0.7406666666666667 vFLOW_Electrolyzer_E[5] == 0


In [32]:
println(smr.constraints[1].constraint_ref[:energy,5])

-vFLOW_SMR_NG[5] + 2.1775180500999998 vFLOW_SMR_H2[5] == 0


In [33]:
println(smr.constraints[1].constraint_ref[:emissions,5])

-0.181048235160161 vFLOW_SMR_NG[5] + vFLOW_SMR_CO2[5] == 0


In [34]:
println(smr_ccs.constraints[1].constraint_ref[:energy,5])

-vFLOW_SMR_CCS_NG[5] + 2.1775180500999998 vFLOW_SMR_CCS_H2[5] == 0


In [35]:
println(smr_ccs.constraints[1].constraint_ref[:emissions,5])

-0.018104823516016097 vFLOW_SMR_CCS_NG[5] + vFLOW_SMR_CCS_CO2[5] == 0


In [36]:
println(smr_ccs.constraints[1].constraint_ref[:captured_emissions,5])

vFLOW_SMR_CCS_CO2_Captured[5] - 0.16294341164414491 vFLOW_SMR_CCS_NG[5] == 0


In [37]:
println(electric_dac.constraints[1].constraint_ref[:captured_co2,5])

vFLOW_Electric_DAC_CO2_Captured[5] - vFLOW_Electric_DAC_CO2_input[5] == 0


In [38]:
println(electric_dac.constraints[1].constraint_ref[:energy,5])

4.38 vFLOW_Electric_DAC_CO2_Captured[5] - vFLOW_Electric_DAC_E[5] == 0


In [39]:
using Gurobi

In [40]:
Macro.set_optimizer(model,Gurobi.Optimizer)

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2024-03-14


In [41]:
Macro.optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 289091 rows, 280348 columns and 731340 nonzeros
Model fingerprint: 0x44164a23
Coefficient statistics:
  Matrix range     [1e-04, 7e+02]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 2e+04]
Presolve removed 188500 rows and 197268 columns
Presolve time: 0.27s
Presolved: 100591 rows, 83080 columns, 266733 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 9
 AA' NZ     : 2.012e+05
 Factor NZ  : 1.735e+06 (roughly 90 MB of memory)
 Factor Ops : 3.105e+07 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual     

The installed electrolyzer capacity is:

In [42]:
Macro.value(Macro.capacity(electrolyzer.TEdges[:H2]))

0.0

The installed SMR capacity is

In [43]:
Macro.value(Macro.capacity(smr.TEdges[:H2]))

1039.5627

The installed SMR CCS capacity is

In [44]:
Macro.value(Macro.capacity(smr_ccs.TEdges[:H2]))

0.0

The installed solar capacity in MW is:

In [45]:
Macro.value(Macro.capacity(solar_pv))

0.0

The installed battery capacity in MW is:

In [46]:
Macro.value(Macro.capacity(battery))

100.0

The installed NGCC capacity in MW is:

In [47]:
Macro.value(Macro.capacity(ngcc.TEdges[:E]))

16617.0

The installed NGCC CCS capacity in MW is:

In [48]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[:E]))

0.0

The installed Electric DAC capacity in Tons CO2/h is:

In [49]:
Macro.value(Macro.capacity(electric_dac.TEdges[:CO2_Captured]))

0.0

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [50]:
base_emissions  = Macro.value(sum(Macro.withdrawal(co2_sink)[t] for t in time_interval(CO2)))

3.42974550073138e7

The resulting CO2 removed from the atmosphere by DAC in tonnes are:

In [51]:
atm_CO2_captured  = Macro.value(sum(Macro.injection(co2_source)[t] for t in time_interval(CO2)))

0.0

The net CO2 emissions (CO2 emissions - CO2 removed from the atmosphere by DAC) in tonnes are:

In [52]:
net_emissions = Macro.value(sum(Macro.withdrawal(co2_sink)[t] for t in time_interval(CO2))) - Macro.value(sum(Macro.injection(co2_source)[t] for t in time_interval(CO2)))

3.42974550073138e7

The resulting captured CO2 (DAC + Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [53]:
base_CO2_captured  = Macro.value(sum(Macro.withdrawal(co2_captured_sink)[t] for t in time_interval(CO2)))

0.0

However, if we add a net-zero CO2 cap accounting for both positive emissions from Natural gas power and H2, and negative emissions from DAC:

In [54]:
Macro.@constraint(model,CO2Cap,sum(Macro.withdrawal(co2_sink)[t] - Macro.injection(co2_source)[t] for t in time_interval(CO2)) <= 0);

In [55]:
Macro.optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 289092 rows, 280348 columns and 748860 nonzeros
Coefficient statistics:
  Matrix range     [1e-04, 7e+02]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 2e+04]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.5077394e+09   3.429746e+07   0.000000e+00      0s
    1100    3.5174056e+09   4.609842e+08   0.000000e+00      5s
    2220    3.5174063e+09   4.235046e+08   0.000000e+00     11s
    3120    3.5174068e+09   3.936423e+08   0.000000e+00     16s
    3940    3.5174071e+09   3.687694e+08   0.000000e+00     21s
    4760    3.5174074e+09   3.308622e+08   0.000000e+00     25s
    5730    3.5174076e+09   2.931337e+08   0.000000e+00     31s
    6680    3.5174078e+09   2.57592

Now, the electrolyzer capacity is:

In [56]:
Macro.value(Macro.capacity(electrolyzer.TEdges[:H2]))

0.0

The SMR capacity is:

In [57]:
Macro.value(Macro.capacity(smr.TEdges[:H2]))

26.330700000000093

The SMR CCS capacity is:

In [58]:
Macro.value(Macro.capacity(smr_ccs.TEdges[:H2]))

1013.2319999999999

Now the solar capacity is:

In [59]:
Macro.value(Macro.capacity(solar_pv))

0.0

The installed battery capacity is:

In [60]:
Macro.value(Macro.capacity(battery))

100.0

The installed NG capacity is:

In [61]:
Macro.value(Macro.capacity(ngcc.TEdges[:E]))

916.0

The installed NG CCS capacity is:

In [62]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[:E]))

15701.0

The installed Electric DAC capacity is:

In [63]:
Macro.value(Macro.capacity(electric_dac.TEdges[:CO2_Captured]))

1318.9497716894978

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [64]:
Macro.value(sum(Macro.withdrawal(co2_sink)[t] for t in time_interval(CO2)))

2.1196417017425955e6

The resulting CO2 removed from the atmosphere by DAC in tonnes are:

In [65]:
atm_CO2_captured  = Macro.value(sum(Macro.injection(co2_source)[t] for t in time_interval(CO2)))

2.1196417017426006e6

The net CO2 emissions (CO2 emissions - CO2 removed from the atmosphere by DAC) in tonnes are:

In [66]:
net_emissions = Macro.value(sum(Macro.withdrawal(co2_sink)[t] for t in time_interval(CO2))) - Macro.value(sum(Macro.injection(co2_source)[t] for t in time_interval(CO2)))

-5.122274160385132e-9

The resulting captured CO2 (DAC + Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [67]:
Macro.value(sum(Macro.withdrawal(co2_captured_sink)[t] for t in time_interval(CO2)))

4.146808034918954e7

We see that in a net-zero emission case with negative emissions from DAC, a combination of Natural Gas Power CCS and SMR H2 CCS with DAC is deployed.

Here is the full hourly time series of captured demand:

In [68]:
Macro.value.(Macro.withdrawal(co2_captured_sink))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{Float64}:
 2804.433870894012
 2648.553508656972
 2542.34575730205
 2512.1840575104
 2553.455531091024
 2658.237292133766
 2791.1696063427544
 2904.2335361496243
 3058.693939882224
 3212.5627652004187
    ⋮
 4021.292751310044
 4274.866790870887
 4482.435310286976
 4316.624037861912
 3684.1172178108063
 3481.0502719923
 3286.262124221832
 3087.5705807754784
 2901.769436244168

In [69]:
#### We need to use dictionaries point at the edge ID, rather than vectors, but for now: 

The outflow of electricity from ngcc_ccs is:

In [69]:
Macro.value.(Macro.flow(ngcc_ccs.TEdges[:E]))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{Float64}:
  7850.0
  7424.0
  7107.0
  6947.0
  6922.0
  7045.0
  7307.0
  7544.0
  7946.0
  8340.0
     ⋮
 10469.0
 11228.0
 11908.0
 11562.0
  9923.0
  9461.0
  9018.0
  8551.0
  8089.0

The inflow of natural gas into ngcc_ccs is:

In [70]:
Macro.value.(Macro.flow(ngcc_ccs.TEdges[:NG]))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{Float64}:
 17093.516693284997
 16165.894003942398
 15475.620782060698
 15127.2178940447
 15072.779942792198
 15340.614662954498
 15911.1243920807
 16427.196169954397
 17302.558426094598
 18160.500537834
     ⋮
 22796.436466496896
 24449.172666522798
 25929.884940590797
 25176.463695256196
 21607.511611142298
 20601.498271996097
 19636.857775801796
 18619.956846405097
 17613.943507258897

The outflow of CO2 (emission) from ngcc_ccs is:

In [71]:
Macro.value.(Macro.flow(ngcc_ccs.TEdges[:CO2]))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{Float64}:
 309.47510299999993
 292.68065791999993
 280.1833830599999
 273.87561026
 272.89002075999997
 277.7391211
 288.06809905999995
 297.4114875199999
 313.2597666799999
 328.79265719999995
   ⋮
 412.7254590199999
 442.6479562399999
 469.4559906399999
 455.8154319599999
 391.2001843399999
 372.9864903799999
 355.52184443999994
 337.1110325799999
 318.8973386199999

The outflow of CO2 (captured) from ngcc_ccs is:

In [72]:
Macro.value.(Macro.flow(ngcc_ccs.TEdges[:CO2_Captured]))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{Float64}:
 2785.275927
 2634.12592128
 2521.65044754
 2464.8804923400003
 2456.01018684
 2499.6520899
 2592.6128915400004
 2676.70338768
 2819.33790012
 2959.1339148000006
    ⋮
 3714.52913118
 3983.8316061600003
 4225.103915760001
 4102.33888764
 3520.8016590600005
 3356.87841342
 3199.69659996
 3033.9992932200003
 2870.07604758